In [1]:
import sys
sys.path.append("src")
from launcher import coso

In [2]:
coso(file="tests/check/test_2.txt", visual="src/VisCoSo/prova.html")

Running solver...
Solution: 2880 (12 subproblems)


In [ ]:
import sys
sys.path.append("src")
from launcher import viscoso

viscoso(file="tests/check/test_2.txt")


```{raw} html
:file: src/VisCoSo/prova.html
```

In [12]:
import sys
sys.path.append("src")
from VisCoSo_widget import viscoso
viscoso()

HTML(value='\n            <script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js…

Button(description='Solve', icon='check', style=ButtonStyle(), tooltip='Solve')

VisCoSoWidget()